# ホールドアウト法によるサポートベクトルマシンの性能評価

---
## 目的
ホールドアウト法を用いて，サポートベクトルマシン（Support Vector Machine; SVM）の性能を評価する．
また，テキストファイルの読み込みやNumpyモジュールを使用した，データセットの読み込み・整理を行う．

## 対応するチャプター
* 5.3: ホールドアウト法
* 5.7: サポートベクトルマシン

## データセットのダウンロード
プログラムの動作に必要なデータをダウンロードし，zipファイルを解凍します．
zipファイルを解凍すると,`./data`ディレクトリの中に`car.txt`と`human.txt`の2つのテキストファイルが保存されています．
下記のプログラムでは，これらのテキストファイルを読み込み，実験に使用します．

In [ ]:
!wget -q http://www.mprg.cs.chubu.ac.jp/tutorial/ML_Lecture/sklearn/data.zip -O data.zip
!unzip -q -o data.zip
!ls ./data

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．
実験にはNumpyとPythonの機械学習用ライブラリである，scikit-learnを使用します．
使用するクラス，関数は以下の通りです．
* `numpy`は配列を扱うためのライブラリ
* `matplotlib`はグラフを描画するためのライブラリ
* `LinearSVC`はサポートベクトルマシンを使用するためのクラス
* `train_test_split`はデータを学習用とテスト用のデータに分割するための関数
* `accuracy_score`は認識率を計算するための関数です．

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## データセットの読み込み
上でダウンロード・解凍したファイルを読み込みます．
Pythonには「リスト」と言う配列に似たものが標準で実装されていますが，
今回はNumpyを利用してデータを読み込みます．

### テキストファイルの読み込み
まず，`open()`関数を使用して，テキストファイルをPython上で読み込みます．

### Numpy array（配列）へ格納
次に，ファイルの中身を参照して配列に入れます．

まず，読み込んだデータを一時的に保存するためのリスト (`car` or `human`) を作成します．

そして，`for line in in_txt1:`のfor文によって，テキストファイルを1行ずつ読み込んで処理を実行します．
このfor文では，「`in_txt1`の中身を変数`line`に1行ずつ入れて実行」というループを実行しています．
このfor文の書き方はPython特有の書き方です．

続いて，`(line.strip()).split('\t')`の`line.strip()`は「文字列変数`line`から空白文字や改行コードを取り除く」という処理です．
これを行わずに処理を行うと，変数`line`に残っている改行コードが一緒に処理されてしまうため，変数やリストに改行コードが入ってしまうことになり注意が必要です．
さらに続けて`.split('\t')`と記述することにより，先程得られた改行コードのない文字列を「指定された文字列で区切る」という処理を行います．
ここでは`'\t'`つまりタブが指定されているため，タブで区切ります．
例えば，`1512.000000 (タブ) 26.779374`という文字列があったとき，この処理を行うことで`['1512.000000', '26.779374']`というリストが生成されます．

最後に，for文を抜けた後に，`np.asarray()`関数を用いて，リストに格納した数値データをnumpyの配列に変換しています．

#### 補足：リスト内包表記
上記の一連の処理を，1行で記述することができる．
このような書き方を「リスト内包表記」と呼び，リストを生成する際によく用いられる．
```
car = np.asarray([(line.strip()).split('\t') for line in in_txt1], dtype=float)
human = np.asarray([(line.strip()).split('\t') for line in in_txt2], dtype=float)
```

In [ ]:
# テキストファイルの読み込み
in_txt1 = open('./data/car.txt')
in_txt2 = open('./data/human.txt')


# Numpy arrayへ格納
car = []
for line in in_txt1:
    car.append((line.strip()).split('\t'))
car = np.asarray(car, dtype=float)

human = []
for line in in_txt2:
    human.append((line.strip()).split('\t'))
human = np.asarray(human, dtype=float)


# 格納したデータの配列サイズを確認
print(car.shape)
print(human.shape)

## データのラベル付けと結合
学習を行う前に各データがcarまたはhumanどちらのクラスに属するかラベル付けをする必要があります．
ここでは，carのラベルを0，humanのラベルを1として，教師ラベルを作成します．
また，この後の処理のために，carとhumanの配列を1つの配列に結合する．

### ラベル付け
`np.zeros`と`np.ones`はそれぞれnumpyの関数です．
`zeros`は指定された数だけ要素を持った配列を用意する関数です．この時，配列の中身は全て0になります．
`ones`は配列の中身は全て1になります．

`car_y`にはcarのデータ数と同じ要素数の0配列が，`human_y`にはhumanと同じ要素数の1配列が生成されます．


### データの結合
データと教師ラベルを結合しますに
結合には，`np.r_[]`関数を使用します．
この関数は，与えられた2つ以上の配列を1つの配列として結合します．
`x = np.r_[car, human]`の場合，配列`car`と`human`を1つの配列`x`として作成します．

最終的に，carとhumanのデータが入った配列`x`と，そのラベルが入った配列`y`が作成されます．
`x`のn番目の要素がどちらのクラスに属するかは配列`y`のn番目の要素を確認すればわかるということになります．

![array.png](https://qiita-image-store.s3.amazonaws.com/0/143078/6cbc837a-d3a4-b953-683e-520052288ffd.png)


確認のため，最後に`print(x.shape, y.shape)`で各配列の要素数を出力します．

In [ ]:
# ラベル付け
car_y = np.zeros(car.shape[0])
human_y = np.ones(human.shape[0])

# 結合
x = np.r_[car, human]
y = np.r_[car_y, human_y]

# 結合したデータの配列サイズを確認
print(x.shape, y.shape)

## データの分割
上記で読み込み・作成したデータを学習用データとテストデータに分割します．

データの分割には`train_test_split`関数を使用します．
はじめに`test_sample_ratio`でテストに用いるデータ数の割合を指定します．
その後，`train_test_split`関数を用いて指定した割合でデータを分割します．
`random_state`はデータをランダムに分割する際のseedです．
seedを変更，または指定しないことで，無作為にデータを分割することが可能です．

In [ ]:
# テストデータの割合を指定
test_sample_ratio = 0.3

# データを分割
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_sample_ratio, random_state=0)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

## SVMの学習
分割した学習用データでSVMを学習し，テストデータで学習したSVMの識別性能を評価します．

まず，`LinearSVC`クラスを実行することでSVMを実行するための準備をします．
`C`はSVMの学習時に用いられるパラメータ (ペナルティパラメータ) です．

その後，`fit`関数を用いることでSVMの学習を実行します．

In [ ]:
# SVMの準備
classifier = LinearSVC(C=1.0, random_state=0)

# 学習
classifier.fit(x_train, y_train)

## SVMの評価
学習したSVMでテストデータを識別することで，性能の確認を行います．

`predict`関数を実行することで，関数に入力したデータの識別結果を出力します．

その後，`accuracy_score`関数を用いることで，識別したクラスと教師ラベルから認識率を算出します．

In [ ]:
# 評価
pred_train = classifier.predict(x_train)
pred_test = classifier.predict(x_test)

score_train = accuracy_score(pred_train, y_train)
score_test = accuracy_score(pred_test, y_test)

print("train accuracy:", score_train)
print("test accuracy:", score_test)

## グラフの描画
識別結果を可視化するために，学習した識別器を用いてグラフを作成する．
どちらのクラスがどの領域かわかりやすく表示することができる．

ここでは`meshgrid`を用いることで，指定した領域を格子状に分割し，その各格子の中心座標を`xx`と`yy`に保存している．
その保存した座標を`classifier.predict()`関数に入力することで，各座標がどちらのクラスに属するかを求めている．
各座標が属するクラスに従って塗りつぶすことで各クラスに属する領域（境界線）を描画している．
塗りつぶしには`contourf`を用いている．

最後に`scatter`関数を用いて，carおよびhumanクラスのデータを散布図としてプロットしている．

In [ ]:
# 作図領域の設定
fig = plt.figure()
subfig = fig.add_subplot(1,1,1)
plt.xlim(0, 10000)
plt.ylim(20, 50)

# 格子点の作成
xx, yy = np.meshgrid(np.linspace(plt.xlim()[0], plt.xlim()[1], 500),
                     np.linspace(plt.ylim()[0], plt.ylim()[1], 500))

# 格子点座標のクラス識別と塗りつぶし
Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
cs = plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)

# データのプロット（散布図）
subfig.scatter(car[:,0], car[:,1],color='black')
subfig.scatter(human[:,0], human[:,1],color='red')

# 軸ラベルの設定
subfig.set_title('Support Vector Machine')
subfig.set_xlabel('Area')
subfig.set_ylabel('complexity')

# 表示
plt.show()

## 課題
以下の課題に取り組みましょう．

1. ホールドアウト法によって分割する学習・テストデータの割合を変更して，認識率の変化を確認してみましょう．
2. SVMのパラメータ`C`を変更して，認識率の変化を確認してみましょう．